In [1]:
import numpy as np
import pandas as pd

In [2]:
dp1 = pd.read_csv("../data/meinian_round1_data_part1_20180408.txt", sep="$")
dp2 = pd.read_csv("../data/meinian_round1_data_part2_20180408.txt", sep="$")

In [10]:
dp = dp1.append(dp2)

In [4]:
dp = dp.sort_values('vid').reset_index(drop=True)

In [5]:
cols = dp['table_id'].unique()
print (len(cols))

2795


In [6]:
len(dp1['table_id'].unique())+len(dp2['table_id'].unique())

2800

In [7]:
dp.shape

(8104368, 3)

# 去除空格、全角转半角、全小写

In [8]:

def DBC2SBC(ustring):
    """全角转半角"""
    rstring = ""
    for uchar in ustring:
        inside_code=ord(uchar)
        if inside_code == 12288:                              #全角空格直接转换            
            inside_code = 32 
        elif (inside_code >= 65281 and inside_code <= 65374): #全角字符（除空格）根据关系转化
            inside_code -= 65248
        rstring += chr(inside_code)
    return rstring

dp['field_results'] = dp['field_results'].apply(lambda x: str(x).strip())
dp['field_results'] = dp['field_results'].apply(lambda x: str(x).lower())
dp['field_results'] = dp['field_results'].apply(lambda x: DBC2SBC(x))

# 相同体检项目拼接组合

In [ ]:
# dp_dup = dp.groupby(['vid', 'table_id'], as_index=False).agg('last') # 重复体检项目只取最后一次

In [9]:
def merge_data(data):
    data['field_results'] = data['field_results'].astype(str)
    if data.shape[0]>1:
        merge_df = ";".join(list(data['field_results']))
    else :
        merge_df = data['field_results'].values[0]
    return merge_df

In [10]:
print ("---------去重和组合-----------")
# 区分唯一项目和重复项目
# size()返回dataFrame的重复记录条数
vid_tabid_group = dp.groupby(['vid','table_id']).size().reset_index()
vid_tabid_group['new_index'] = vid_tabid_group['vid'] + '_' + vid_tabid_group['table_id']
vid_tableid_gruop_dup = vid_tabid_group[vid_tabid_group[0]>1]['new_index']

vid_tableid_gruop_dup_list = list(vid_tableid_gruop_dup)
dp['new_index'] = dp['vid'] + '_' + dp['table_id']
dup_part = dp[dp['new_index'].isin(vid_tableid_gruop_dup_list)]
dup_part = dup_part.sort_values(['vid','table_id'])
unique_part = dp[~dp['new_index'].isin(vid_tableid_gruop_dup_list)]

# 仅对重复项目进行拼接操作 
dp_dup = dup_part.groupby(['vid', 'table_id'], as_index=False).apply(merge_data).reset_index() # 重复体检项目合并结果
dp_dup.rename(columns={0:'field_results'}, inplace=True)

dp_dup = pd.concat([dp_dup, unique_part[['vid', 'table_id', 'field_results']]])
print ("---------END-----------")

---------去重和组合-----------
---------END-----------


In [11]:
dp_fmt = dp_dup.set_index(['vid', 'table_id']).unstack(fill_value=None) # 将同一个vid所有项目转换成整行
dp_fmt.columns = dp_fmt.columns.droplevel(level=0)  # 删除cols的第一索引

In [12]:
dp_fmt.shape

(57298, 2795)

In [13]:
dp_fmt.to_csv('../Output/0504/data_yuchuli.csv', encoding='utf-8')

以上为数据初处理：
    将所有记录转换成以vid为主键，table_id为字段的格式

——————————————————————————————————————————————————————————————————

In [1]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

dp_fmt = pd.read_csv("../Output/0504/data_yuchuli.csv", encoding='utf-8')

In [2]:
dp_fmt.shape

(57298, 2796)

# 字段筛选，缺失值占XX%以上的字段舍弃

In [3]:
def remainFeat(data, thread=0.9, exclude_cols=['0101', '0102', '0104', '0120', '0121','0125','0130','0131',
                                               '0409','0413', '0424','0434', '0437', '0441', '0449','0438',
                                               '1316','1330','1334','3402','3401',
                                               'A301','A705','A801','B701']):
    drop_cols = []
    num_cols = int(data.shape[0])
    for c in data.columns:
        if c not in exclude_cols:
            num_nan = data[c].isnull().sum()
            if num_nan == 0:
                continue
            if (num_nan / num_cols) > thread:
                drop_cols.append(c)
    remain_feat = [c for c in data.columns if c not in drop_cols]
    return remain_feat

In [4]:
feats = remainFeat(dp_fmt, 0.95)

In [5]:
dp_fmt = dp_fmt[feats]

In [6]:
dp_fmt.shape

(57298, 394)

In [7]:
dp_fmt.to_csv('../Output/0504/data_tezheng1.csv', index=False, encoding='utf-8')

以上为数据再处理：
    由于数据中 体检项目的量较大，故2795个字段数量略长，而且其中很多很多字段大部分都是NAN，故舍弃，不考虑进来

——————————————————————————————————————————————————————————————————

In [1]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

dp_fmt = pd.read_csv("../Output/0504/data_tezheng1.csv", encoding='utf-8')

In [2]:
train_df = pd.read_csv("../data/meinian_round1_train_20180408.csv", encoding='gbk')
test_df = pd.read_csv("../data/meinian_round1_test_b_20180505.csv", encoding='gbk')

In [3]:
dp_fmt.shape

(57298, 394)

In [4]:
# dp_fmt = dp_fmt.drop('Unnamed: 0', axis=1)

In [4]:
train = pd.merge(train_df, dp_fmt, on='vid', how='left')
test = pd.merge(test_df, dp_fmt, on='vid', how='left')

# 清洗train的目标指标   

收缩压、舒张压、血清甘油三酯、血清高密度脂蛋白、血清低密度脂蛋白

In [5]:
def cleanLabelForDigit(label):
    x = str(label)
    if '弃查' in x or '未查' in x:
        return np.nan
    elif len(x)>3:
        return x[:3]
    return x

def cleanLabelForNotDigit(label):
    import re
    p = re.compile('\d+\.?\d*')
    x = str(label)
    if '弃查' in x or '未查' in x:
        return np.nan
    elif x.count('.') >= 2: # 2.2.8
        idx = x.index('.')+1
        return (x[:idx]+x[idx:].replace('.', ''))
    elif len(x)>16:  # 4.019999999999999
        return x[0:6]
    else:   # 16.04++   > 11.00   7.75轻度
        return p.findall(x)[0]

In [6]:
for c in ['收缩压','舒张压','血清甘油三酯','血清高密度脂蛋白','血清低密度脂蛋白']:
    if c == '收缩压' or c == '舒张压':
        train[c] = train[c].apply(cleanLabelForDigit)
        train[c]=train[c].astype('float64')
    else:
        train[c] = train[c].apply(cleanLabelForNotDigit)
        train[c]=train[c].astype('float64')

In [7]:
train.to_csv('../Output/0505/train_data_tezheng1.csv', index=False, encoding='utf-8')
test.to_csv('../Output/0505/test_data_tezheng1.csv', index=False, encoding='utf-8')

In [ ]:
train

——————————————————————————————————————————————————————————————————